In [1]:
import nltk
import re
import codecs
import csv
import os
from sklearn import metrics

path = 'D:\Dropbox\ongoing\chrisIntervention'

In [10]:
# read cohort info from the CSV and save them with the sbj ID

intvIdx = 'Ex1'

category = {}
with open(os.path.join(path, intvIdx + '_key.csv')) as f:
    sbjData = csv.reader(f)
    next(sbjData) # skip the header
    for row in sbjData:
        category[row[0]] = int(row[1])

subjectId = category.keys()        
        
#print(subjectId)
#print()
print('# of cohort 1: ', sum(1 for sbj in category if category[sbj] == 1))
print('# of cohort 2: ', sum(1 for sbj in category if category[sbj] == 2))


# of cohort 1:  148
# of cohort 2:  197


In [11]:
# build the dictionary based on the training set
# this is just to check if it is working
# we should build the dictionary for each leave-one-out test

def make_dictionary(subjectPool, critNumSbj):
    allWords = []
    for sbj in subjectPool:
        sbjWords = []
        for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
            line = line.strip()
            line = line.lower()
            tokens = nltk.wordpunct_tokenize(line)
            text = nltk.Text(tokens)
            words = [w for w in text if re.search('[a-z]+',w) and len(w)>2]
            sbjWords.extend(words)
        # when making dictionary, make the words from a document unique
        sbjWords = list(set(sbjWords))
        allWords.extend(sbjWords)
    
    # once we make list of all words, we count the words
    cntWords = nltk.FreqDist(allWords)
    #for ii in range(21,0,-2):
    #    print('# words with freq >', ii, ' : ', sum(1 for word in cntWords if cntWords[word] > ii))
    
    # we go with the words that came from at least N different people (arbitrary) 
    # we can change and see what happens ...
    return [word for word in cntWords if cntWords[word] > critNumSbj]

dictionary = make_dictionary(subjectId, 5)

print(dictionary)

['football', 'has', 'more', 'own', 'matter', 'listing', 'two', 'than', 'friends', 'still', 'was', 'bad', 'well', 'favorite', 'out', 'say', 'fun', 'old', 'help', 'sister', 'different', 'hang', 'mind', 'homework', 'much', 'really', 'rap', 'when', 'mad', 'from', 'sence', 'most', 'free', 'times', 'everyday', 'why', 'want', 'art', 'down', 'makes', 'after', 'and', 'learn', 'can', 'value', 'had', 'games', 'gives', 'back', 'team', 'ever', 'new', 'whole', 'just', 'its', 'happy', 'keep', 'stuff', 'let', 'years', 'hard', 'day', 'people', 'others', 'spending', 'friend', 'chose', 'not', 'love', 'anything', 'them', 'because', 'your', 'alone', 'soccer', 'enjoy', 'someone', 'listening', 'have', 'calm', 'then', 'also', 'their', 'sport', 'try', 'the', 'stay', 'done', 'without', 'lots', 'close', 'away', 'last', 'great', 'parents', 'helps', 'these', 'thing', 'things', 'brother', 'living', 'into', 'very', 'home', 'little', 'for', 'other', 'happen', 'feeling', 'sad', 'songs', 'smile', 'nice', 'way', 'troubl

In [12]:
# we define a function that tunrs text into a list of tokens

def extract_feature(text, dictionary):
    text = text.strip()
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)
    feature = {}
    for word in dictionary:
        feature[word] = (word in text)
    return feature

print('nltk output: ', nltk.wordpunct_tokenize('You think you can do this one, Kate?'))
print(extract_feature('You think you can do this one, Kate?', dictionary))

nltk output:  ['You', 'think', 'you', 'can', 'do', 'this', 'one', ',', 'Kate', '?']
{'today': False, 'about': False, 'different': False, 'into': False, 'important': False, 'anything': False, 'creativity': False, 'them': False, 'boring': False, 'are': False, 'wouldn': False, 'many': False, 'enjoying': False, 'football': False, 'only': False, 'very': False, 'home': False, 'most': False, 'friend': False, 'hop': False, 'chose': False, 'school': False, 'that': False, 'thats': False, 'look': False, 'picked': False, 'part': False, 'your': False, 'being': False, 'play': False, 'lot': False, 'funny': False, 'hip': False, 'everyday': False, 'keeps': False, 'everything': False, 'all': False, 'this': True, 'make': False, 'more': False, 'inportant': False, 'see': False, 'where': False, 'time': False, 'long': False, 'basketball': False, 'nothing': False, 'why': False, 'alone': False, 'for': False, 'been': False, 'playing': False, 'world': False, 'everyone': False, 'other': False, 'happen': False, 'a

In [5]:
# leave-one-out classification

predLabel = []
trueLabel = []
currSbj = 1;

for sbj in subjectId:
    # sbj is being tested, others are used to train the classifier
    trainSbj = list(subjectId)
    trainSbj.remove(sbj)
    # create the dictionary
    trainDict = make_dictionary(trainSbj, 5)
    trainText = []
    # create the training set
    for tSbj in trainSbj:
        tmpText = ''
        for line in codecs.open(os.path.join(path, intvIdx, tSbj + '.txt'), 'r' , 'utf-8'):
            tmpText = tmpText + ' ' + line
        trainText.append([extract_feature(tmpText, trainDict), category[tSbj]])
    # train classifier
    classifier = nltk.NaiveBayesClassifier.train(trainText)

    # now the test set
    trueLabel.append( category[sbj] )
    testText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        testText = testText + ' ' + line
    # predict!
    currPred = classifier.classify(extract_feature(testText, trainDict))
    predLabel.append( currPred )
    
    print(str(currSbj) + ': pred - ' + str(currPred) + ', actual - ' + str(category[sbj]) )
    currSbj = currSbj + 1

print('done. ')


1: pred - 1, actual - 1
2: pred - 2, actual - 2
3: pred - 1, actual - 2
4: pred - 1, actual - 2
5: pred - 1, actual - 1
6: pred - 2, actual - 2
7: pred - 2, actual - 1
8: pred - 2, actual - 2
9: pred - 2, actual - 1
10: pred - 2, actual - 1
11: pred - 2, actual - 2
12: pred - 2, actual - 2
13: pred - 1, actual - 1
14: pred - 2, actual - 1
15: pred - 2, actual - 2
16: pred - 2, actual - 2
17: pred - 2, actual - 2
18: pred - 1, actual - 1
19: pred - 2, actual - 2
20: pred - 2, actual - 2
21: pred - 2, actual - 1
22: pred - 2, actual - 2
23: pred - 1, actual - 1
24: pred - 2, actual - 2
25: pred - 1, actual - 1
26: pred - 1, actual - 1
27: pred - 2, actual - 2
28: pred - 2, actual - 2
29: pred - 2, actual - 2
30: pred - 1, actual - 2
31: pred - 2, actual - 2
32: pred - 1, actual - 1
33: pred - 2, actual - 1
34: pred - 1, actual - 1
35: pred - 2, actual - 2
36: pred - 2, actual - 2
37: pred - 1, actual - 2
38: pred - 2, actual - 2
39: pred - 2, actual - 2
40: pred - 2, actual - 1
41: pred 

In [6]:
# ok, it is POSSIBLE TO PREDICT cohort from the writings.

print(metrics.classification_report(trueLabel, predLabel))


             precision    recall  f1-score   support

          1       0.72      0.55      0.62       148
          2       0.71      0.84      0.77       197

avg / total       0.72      0.72      0.71       345



In [9]:
# now, show me the informative features

dictionary = make_dictionary(subjectId, 5)

allText = []

for sbj in subjectId:
    tmpText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        tmpText = tmpText + ' ' + line
        allText.append([extract_feature(tmpText, dictionary), category[sbj]])

# train classifier based on all information
classifier = nltk.NaiveBayesClassifier.train(allText)

# and show me the informative features
classifier.show_most_informative_features(50)




Most Informative Features
                  values = True                1 : 2      =     12.8 : 1.0
              everything = True                1 : 2      =     12.8 : 1.0
                   never = True                1 : 2      =      5.9 : 1.0
                    into = True                1 : 2      =      5.8 : 1.0
                     old = True                1 : 2      =      5.8 : 1.0
                     now = True                1 : 2      =      5.6 : 1.0
                     see = True                1 : 2      =      5.6 : 1.0
                    dont = True                1 : 2      =      5.1 : 1.0
                    such = True                1 : 2      =      4.9 : 1.0
                    same = True                1 : 2      =      4.9 : 1.0
                 writing = True                1 : 2      =      4.9 : 1.0
                 trouble = True                1 : 2      =      4.9 : 1.0
                     rap = True                1 : 2      =      4.9 : 1.0